# Imports

In [1]:
from pyiron_contrib.tinybase.task import AbstractTask, FunctionTask, SeriesTask, LoopTask

/home/poul/pyiron/contrib/pyiron_contrib/__init__.py:9: UserWarning: pyiron module not found, importing Project from pyiron_base
  warnings.warn("pyiron module not found, importing Project from pyiron_base")


In [2]:
from pyiron_contrib.tinybase.executor import ProcessExecutor, BackgroundExecutor, Executor

In [3]:
import logging
logging.getLogger().setLevel(20)

In [4]:
import numpy as np

# Function Task

## Basic

In [5]:
def calc_fib(n):
    import time
    n1 = n2 = 1
    for i in range(n):
        time.sleep(.1)
        x = n1 + n2
        n1 = n2
        n2 = x
    return x

In [6]:
f = FunctionTask(calc_fib)

In [7]:
f.input.storage

DataContainer([])

In [8]:
f.input.args

[]

In [9]:
f.input.kwargs

{}

In [10]:
f.input.kwargs['n'] = 10

In [11]:
f.input.kwargs

{'n': 10}

In [12]:
f.execute()

(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fa84748f310>)

## We can use an executor to distribute the task to any compute resource

### Directly in the foreground

In [13]:
exe = Executor().submit([f])

In [14]:
exe.run()

In [15]:
exe.status[0]

ReturnStatus(Code.DONE, None)

In [16]:
exe.output[0].result

144

### Do the same but in the background

In [17]:
f = FunctionTask(calc_fib)

In [18]:
f.input.kwargs['n'] = 100

In [19]:
exe = BackgroundExecutor(max_threads=1).submit([f])

In [20]:
exe.run()

In [21]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [22]:
exe.wait()

In [23]:
exe.output[0].result

927372692193078999176

### Do the same but in the background as process

In [24]:
f = FunctionTask(calc_fib)

In [25]:
f.input.kwargs['n'] = 100

In [26]:
exe = ProcessExecutor(max_processes=1).submit([f])

In [27]:
exe.run()

In [28]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [29]:
exe.wait()

In [30]:
exe.output[0].result

927372692193078999176

In [31]:
exe._run_machine.state

<Code.FINISHED: 'finished'>

# Executors handle single Tasks and lists of them on the same footing

In [32]:
tasks = [FunctionTask(calc_fib) for _ in range(10)]

In [33]:
for i, n in enumerate(tasks):
    n.input.kwargs['n'] = 3 + i

## With the basic executor

In [34]:
exe = Executor().submit(tasks)
exe.run()

In [35]:
exe.output

(<pyiron_contrib.tinybase.task.FunctionOutput at 0x7fa8472f55a0>,
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fa8472f6890>)

In [36]:
exe.output[1].result

8

## With the process executor

In [37]:
exe = ProcessExecutor(max_processes=4).submit(tasks)
exe.run()

In [38]:
exe.wait()

In [39]:
exe.status

[ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None)]

In [40]:
exe.output

In [41]:
exe.output[5].result

55

# SeriesTask

In [42]:
s = SeriesTask()

In [43]:
f1 = FunctionTask(calc_fib)

In [44]:
f2 = FunctionTask(np.sqrt)

In [45]:
def transfer(input, output):
    input.args = [output.result]

In [46]:
s.input.first(f1).then(f2, transfer)

In [47]:
s.input.tasks[0].input.kwargs['n'] = 10

In [48]:
status, output = s.execute()

In [49]:
status

ReturnStatus(Code.DONE, None)

In [50]:
output.result

12.0

# Loop Task

## Simple repeat loop

In [56]:
l = LoopTask()

In [57]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [58]:
l.input.repeat(10, restart=lambda output, input, scratch: print(output.result))

In [59]:
l.execute()

0.9662652321813043
0.27235911125432555
0.11494861653393462
0.8032152774735889
0.3227518840778105
0.6684733014540742
0.39643556152196313
0.8518645269264987
0.2961018352625848


(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fa847309fc0>)

In [66]:
exe = BackgroundExecutor(max_threads=1).submit([l])
exe.run()
exe.wait()

In [61]:
exe.output[0].result

0.385781341996812

## Loop with a termination condition

In [62]:
l = LoopTask()

In [63]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [64]:
l.input.control_with(
    condition=lambda task, output, scratch: output.result < .15,
    restart=lambda output, input, scratch: print(output.result)
)

In [65]:
l.execute()

0.3274925046984414
0.9161078074778493
0.5297212077927368
0.9890799661513604
0.42191837857926595
0.8488251952343272
0.9693005918735547
0.44102348613413955
0.18869549585034684
0.3953730135658532
0.15772497922300788
0.8254814730759052
0.6498952850955698
0.588999434645353
0.6858310499583608
0.6020208768884394
0.39978751512947674
0.7660632855985916
0.18473064026263886
0.2630763951239491
0.6478896023670242


(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fa847309000>)

# Implementation Examples

In [67]:
from pyiron_contrib.tinybase.task import TaskGenerator, FunctionTask, ReturnStatus
from pyiron_contrib.tinybase.container import AbstractInput, AbstractOutput, StorageAttribute
import time

class WaitInput(AbstractInput):
    time = StorageAttribute().type(float).default(lambda: 10.0)
    n = StorageAttribute().type(int).default(lambda: 10)

class WaitOutput(AbstractOutput):
    pass

class WaitGenerator(TaskGenerator):
    def _get_input(self):
        return WaitInput()
    def _get_output(self):
        return WaitOutput()
    def __iter__(self):
        tasks = []
        for _ in range(self.input.n):
            tasks.append(t := FunctionTask(time.sleep))
            t.input.args = [self.input.time]
        ret, out = zip(*(yield tasks))
        return ReturnStatus.done(), self._get_output()

In [68]:
%%time
wait = WaitGenerator(capture_exceptions=False)
wait.input.time = 2.0
wait.execute()

CPU times: user 6.81 ms, sys: 1.68 ms, total: 8.49 ms
Wall time: 20 s


(ReturnStatus(Code.DONE, None), <__main__.WaitOutput at 0x7fa8474c2dd0>)

In [69]:
from pyiron_contrib.tinybase.executor import BackgroundExecutor

In [73]:
%%time
exe = ProcessExecutor(max_processes=1).submit([wait])
exe.run()
exe.wait()

CPU times: user 24 ms, sys: 25.8 ms, total: 49.8 ms
Wall time: 20.1 s


In [76]:
%%time
exe = ProcessExecutor(max_processes=4).submit([wait])
exe.run()
exe.wait()

CPU times: user 28.2 ms, sys: 41.8 ms, total: 69.9 ms
Wall time: 6.08 s


In [75]:
%%time
exe = BackgroundExecutor(max_threads=4).submit([wait])
exe.run()
exe.wait()

CPU times: user 11.7 ms, sys: 3.65 ms, total: 15.3 ms
Wall time: 6.03 s
